### Importazione librerie

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from ipywidgets import interact 
import numpy as np
from itertools import product

In [ ]:
dati = pd.read_csv('Bitcoin (€) da yfinance dal 17-09-2014 al 24-04-2024.csv')
dati = dati[['Date', 'Close']]
dati = dati.rename(columns = {'Date':'Timestamp', 'Close': 'Price'})
dati['Timestamp'] = pd.to_datetime(dati['Timestamp'])
dati['Timestamp'] = dati['Timestamp'].dt.strftime('%Y-%m-%d')
dati['Timestamp'] = pd.to_datetime(dati['Timestamp'], format='%Y-%m-%d')
dati

In [ ]:
data_inizio = '2021-02-01'
data_fine = '2021-07-01'
indice_inizio = dati[dati['Timestamp'] == data_inizio].index[0]
indice_fine = dati[dati['Timestamp'] == data_fine].index[0]
dati = dati[indice_inizio:indice_fine + 1].reset_index(drop=True)
dati

In [ ]:
# Parametri fissi
liquidità_iniziale = 0  # Liquidità iniziale in euro
bitcoin_iniziali = 0.02478146   # Quantità iniziale di Bitcoin posseduti
prezzo_medio_acquisti = 15000  # Prezzo medio ponderato degli acquisti
prezzo_medio_vendite_iniziale = 20000   # Prezzo medio ponderato delle vendite

# Parametri variabili
soglia_sopportabile = 0.15  # Percentuale di discesa sopportabile (es. 20%)
vendita_rispetto_inizio = True  # Se vendere rispetto ai BTC iniziali
paracadute_variabile = True
sopportazione_variabile = True
pct_paracadute = 0  # Percentuale di paracadute (ad es. 5%)
# Con A=1 e B=1 la mappatura torna a essere l'identità
A = 0
B = 0

# Parametri calcolati
# Calcolo la base di riferimento per il prezzo (prezzo medio vendite + pct_paracadute)
prezzo_paracadute = prezzo_medio_vendite_iniziale * (1 + pct_paracadute)

In [ ]:
def grafico(dati, prezzo_paracadute):
    # Creazione del grafico
    plt.figure(figsize=(14, 7))
    plt.plot(dati["Timestamp"], dati["Price"], color="blue", label="Prezzo Bitcoin")

    # Linea verde per il prezzo medio di acquisto
    plt.axhline(y=prezzo_medio_acquisti, color="green", linestyle="--", linewidth=1.5, label="Prezzo Medio Pesato Acquisto Iniziale")

    # Linea rossa per il prezzo medio di vendita
    plt.axhline(y=prezzo_medio_vendite_iniziale, color="red", linestyle="--", linewidth=1.5, label="Prezzo Medio Pesato Vendita Iniziale")

    # Linea rossa per il prezzo medio di vendita
    plt.axhline(y=prezzo_paracadute, color="purple", linestyle="--", linewidth=1.5, label="Prezzo Paracadute")

    # Dettagli del grafico
    plt.title("Andamento del Prezzo di Bitcoin con soglie di Acquisto, Vendita e Paracadute")
    plt.xlabel("Data")
    plt.ylabel("Prezzo in Euro")
    plt.legend()
    plt.grid()

    # Mostra il grafico
    plt.show()

In [ ]:
grafico(dati, prezzo_paracadute)

### Strategia

In [ ]:
def calcola_max_drawdown_relativo_prezzo(prezzi, prezzi_paracadute):
    """
    Calcola il massimo drawdown dei prezzi, cioè la massima discesa percentuale del prezzo
    rispetto al massimo precedente, ma relativamente al prezzo paracadute

    Parameters:
    - prezzi (pd.Series or list): Serie temporale dei prezzi.

    Returns:
    - max_drawdown (float): Il massimo drawdown percentuale sui prezzi.
    - drawdown_serie (pd.Series): La serie temporale del drawdown percentuale giorno per giorno.
    """
    # Trasformo i prezzi in una Serie pandas se non lo è già
    prezzi = pd.Series(prezzi)
    
    # Calcola i massimi cumulativi dei prezzi fino a ogni punto
    max_prezzi_cumulativi = prezzi.cummax()
    
    # Calcola il drawdown percentuale relativo
    drawdown_serie = (max_prezzi_cumulativi - prezzi) / (max_prezzi_cumulativi - prezzi_paracadute)
    
    # Trova il massimo drawdown
    max_drawdown = drawdown_serie.max()
    
    # Ritorna il max drawdown
    return max_drawdown

def mappatura_discesa_vendita(percentuale_discesa_relativa, A, B):
    return percentuale_discesa_relativa ** A / (B + (1 - B) * percentuale_discesa_relativa ** A)

In [ ]:
# # Funzione per generare il grafico interattivo
# def grafico_interattivo(A, B):
#     # Definisci il range di x (percentuale_discesa_relativa)
#     x = np.linspace(0, 1, 100)  # da 0% a 100% di discesa relativa

#     # Calcola i valori y in base ai parametri A e B
#     y = mappatura_discesa_vendita(x, A, B)

#     # Crea il grafico
#     fig = go.Figure()
#     fig.add_trace(go.Scatter(
#         x=x,
#         y=y,
#         mode='lines',
#         line=dict(color='blue', width=2),
#         name=f"Curva con A={A}, B={B}"
#     ))

#     # Aggiunge dettagli al layout
#     fig.update_layout(
#         title=f"Mappatura Discesa-Vendita con A={A} e B={B}",
#         xaxis_title="Percentuale Discesa Relativa",
#         yaxis_title="Mappatura Discesa Vendita",
#         template="plotly_white",
#         yaxis=dict(range=[0, 1]),  # Imposta il range y per una scala uniforme
#         xaxis=dict(range=[0, 1])   # Imposta il range x per percentuali da 0 a 1
#     )

#     # Mostra il grafico
#     fig.show()

In [ ]:
# # Usa interattività per gli slider
# interact(grafico_interattivo, A=(0.1, 5.0, 0.1), B=(0.0, 1.0, 0.05))

In [ ]:
def strategia_vendita(dati, liquidità_iniziale, bitcoin_iniziali, prezzo_medio_vendite_iniziale, soglia_sopportabile, vendita_rispetto_inizio, paracadute_variabile, sopportazione_variabile, pct_paracadute, A, B, stampa_grafico=False):

    prezzo_paracadute = prezzo_medio_vendite_iniziale * (1 + pct_paracadute)

    # Variabili iniziali
    btc_posseduti = bitcoin_iniziali
    liquidità = liquidità_iniziale
    massimo_prezzo_raggiunto = dati['Price'].iloc[0] # Per tenere traccia del massimo raggiunto
    massimo_precedente = dati['Price'].iloc[0]

    prezzi_paracadute = [prezzo_paracadute]

    # DataFrame per registrare le transazioni giornaliere
    transazioni = []

    # Itera sulle righe del DataFrame (supponendo ordinato per data)
    for i in range(1, len(dati)):
        # Aggiorna il massimo prezzo raggiunto
        prezzo_attuale = dati["Price"].iloc[i]

        if max(massimo_prezzo_raggiunto, prezzo_attuale) > massimo_prezzo_raggiunto:
            massimo_precedente = massimo_prezzo_raggiunto
            massimo_prezzo_raggiunto = max(massimo_prezzo_raggiunto, prezzo_attuale)
            if paracadute_variabile:
                prezzo_paracadute = prezzo_paracadute * massimo_prezzo_raggiunto / massimo_precedente #prezzo_attuale / rapporto_prezzo_e_paracadute
                if stampa_grafico:
                    grafico(dati.iloc[:i+1], prezzo_paracadute)

            prezzi_paracadute.append(prezzo_paracadute)

            if sopportazione_variabile:
                max_drawdown_relativo_prezzo = calcola_max_drawdown_relativo_prezzo(dati["Price"].iloc[:i+1], prezzi_paracadute)
                # Cambio la soglia sopportabile solo se il max drawdown relativo che c'è stato in passato è maggiore della soglia sopportabile attuale
                if max_drawdown_relativo_prezzo > soglia_sopportabile:
                    soglia_sopportabile = max_drawdown_relativo_prezzo
        else:
            prezzi_paracadute.append(prezzo_paracadute)

        azione = "-"
        btc_da_vendere = 0
        liquidità_guadagnata = 0
        pct_btc_venduta_attuali = '-'
        pct_btc_venduta_iniziali = '-'

        # Calcola la percentuale attuale di discesa rispetto alla base di riferimento
        percentuale_discesa_relativa = (massimo_prezzo_raggiunto - prezzo_attuale) / (massimo_prezzo_raggiunto - prezzo_paracadute)
        if percentuale_discesa_relativa > 1:
            percentuale_discesa_relativa = 1
        
        # Controlla se il prezzo attuale è in discesa rispetto al prezzo di ieri
        prezzo_ieri = dati["Price"].iloc[i - 1]
        prezzo_in_discesa = prezzo_attuale < prezzo_ieri
        variazione_prezzo = (prezzo_attuale - prezzo_ieri) / prezzo_ieri

        # Condizione di vendita
        if percentuale_discesa_relativa > soglia_sopportabile and prezzo_in_discesa:
            # Calcola la percentuale di BTC da vendere
            percentuale_da_vendere = mappatura_discesa_vendita(percentuale_discesa_relativa, A, B) if mappatura_discesa_vendita(percentuale_discesa_relativa, A, B) <= 1 else 1
            
            if vendita_rispetto_inizio:
                # Calcola la quantità di BTC da vendere rispetto ai BTC iniziali
                btc_da_vendere = bitcoin_iniziali * percentuale_da_vendere
            else:
                # Calcola la quantità di BTC da vendere rispetto ai BTC attualmente posseduti
                btc_da_vendere = btc_posseduti * percentuale_da_vendere

            # Verifica di non vendere più BTC di quanti ne possieda
            if btc_da_vendere > btc_posseduti:
                btc_da_vendere = btc_posseduti

            if btc_posseduti > 0:
                # Effettua la vendita
                liquidità_guadagnata = btc_da_vendere * prezzo_attuale
                liquidità += liquidità_guadagnata
                btc_posseduti_prima = btc_posseduti
                btc_posseduti -= btc_da_vendere
                azione = "Vendita"

                # Calcola le percentuali di BTC venduta
                pct_btc_venduta_attuali = (btc_da_vendere / btc_posseduti_prima)
                pct_btc_venduta_iniziali = (btc_da_vendere / bitcoin_iniziali)

        # Valore totale del portafoglio
        valore_totale_portafoglio = liquidità + (btc_posseduti * prezzo_attuale)

        # Registra i dati nel dataframe delle transazioni
        transazioni.append({
            "Timestamp": dati["Timestamp"].iloc[i],
            "prezzo": prezzo_attuale,
            "max_prezzo": massimo_prezzo_raggiunto,
            "prezzo_paracadute": prezzo_paracadute,
            "soglia_sopportabile": soglia_sopportabile,
            "variazione % prezzo": variazione_prezzo,  # Convertito in percentuale
            "% discesa relativa": percentuale_discesa_relativa if prezzo_in_discesa else "-",
            "azione": azione,
            "quantità BTC venduta": btc_da_vendere,
            "% BTC venduta attuali": pct_btc_venduta_attuali,
            "% BTC venduta iniziali": pct_btc_venduta_iniziali,
            "liquidità guadagnata": liquidità_guadagnata,
            "liquidità": liquidità,
            "bitcoin": btc_posseduti,
            "valore_totale_portafoglio": valore_totale_portafoglio
        })

    # Crea un DataFrame dalle transazioni
    df_transazioni = pd.DataFrame(transazioni)

    # Mostra le prime righe del DataFrame delle transazioni
    return df_transazioni, prezzi_paracadute

In [ ]:
df_transazioni, prezzi_paracadute = strategia_vendita(dati, liquidità_iniziale, bitcoin_iniziali, prezzo_medio_vendite_iniziale, soglia_sopportabile, vendita_rispetto_inizio, paracadute_variabile, sopportazione_variabile, pct_paracadute, A, B)

In [ ]:
df_transazioni['valore_totale_portafoglio'].iloc[-1] / (liquidità_iniziale + bitcoin_iniziali * dati['Price'].iloc[0])

In [ ]:
df_transazioni[:]

In [ ]:
# Supponiamo che df_transazioni sia già creato e contenga tutte le colonne richieste
# Se ci sono NaN, conviene sistemarli per evitare problemi
df_transazioni['quantità BTC venduta'] = pd.to_numeric(df_transazioni['quantità BTC venduta'], errors='coerce')

# Filtro solo le transazioni di vendita
vendite = df_transazioni[df_transazioni['azione'] == 'Vendita']

# Calcolo della dimensione dei punti per le vendite
max_quantita_vendita = vendite['quantità BTC venduta'].max()

def calcola_dimensione(quantita, max_quantita, min_size=10, max_size=50):
    return min_size + (max_size - min_size) * (quantita / max_quantita) if max_quantita != 0 else min_size

# Applica la funzione di dimensione per ogni punto vendita
dimensioni_vendite = vendite['quantità BTC venduta'].apply(lambda x: calcola_dimensione(x, max_quantita_vendita))

# Funzione per formattare in modo sicuro i prezzi e altri valori
def safe_format(value, precision=2):
    return f"{value:.{precision}f}" if pd.notna(value) else "None"

# Creazione del grafico dei prezzi con le vendite
fig = go.Figure()

# Linea dell'andamento dei prezzi
fig.add_trace(go.Scatter(
    x=df_transazioni['Timestamp'],
    y=df_transazioni['prezzo'],
    mode='lines',
    name='Prezzi Bitcoin',
    line=dict(color='blue', width=2),
    hoverinfo='x+y'
))

# Scatter plot per le vendite con dimensione in base alla quantità BTC venduta
fig.add_trace(go.Scatter(
    x=vendite['Timestamp'],
    y=vendite['prezzo'],
    mode='markers',
    name='Vendite',
    marker=dict(color='red', size=dimensioni_vendite, opacity=0.7),
    text=[
        f"Timestamp: {timestamp}<br>Prezzo: {safe_format(prezzo)} €<br>Variazione % prezzo: {safe_format(var_prezzo * 100, 2)} %<br>% discesa relativa: {round(desc * 100, 2) or 'None'} %<br>Quantità BTC venduta: {safe_format(qta, 4)}<br>% BTC venduta attuali: {safe_format(pct_attuali * 100, 2)} %<br>% BTC venduta iniziali: {safe_format(pct_iniziali * 100, 2)} %<br>Liquidità guadagnata: {safe_format(liquid_guad, 2)} €<br>Liquidità totale: {safe_format(liquid_tot, 2)} €<br>BTC rimanenti: {safe_format(btc_posseduti, 4)}<br>Valore totale portafoglio: {safe_format(valore_port, 2)} €"
        for timestamp, prezzo, var_prezzo, desc, qta, pct_attuali, pct_iniziali, liquid_guad, liquid_tot, btc_posseduti, valore_port in zip(
            vendite['Timestamp'],
            vendite['prezzo'],
            vendite['variazione % prezzo'],
            vendite['% discesa relativa'],
            vendite['quantità BTC venduta'],
            vendite['% BTC venduta attuali'],
            vendite['% BTC venduta iniziali'],
            vendite['liquidità guadagnata'],
            vendite['liquidità'],
            vendite['bitcoin'],
            vendite['valore_totale_portafoglio']
        )
    ],
    hoverinfo='text'
))

# Aggiunta delle linee orizzontali
fig.add_hline(y=prezzo_medio_acquisti, line=dict(color='green', width=2, dash='dash'), name='Prezzo Medio Acquisto')
fig.add_hline(y=prezzo_medio_vendite_iniziale, line=dict(color='red', width=2, dash='dash'), name='Prezzo Medio Vendita')
for prezzo_paracadute in pd.Series(prezzi_paracadute).unique():
    fig.add_hline(y=prezzo_paracadute, line=dict(color='purple', width=2, dash='dash'), name='Base di Riferimento')

# Configurazione del layout con range slider sull'asse X
fig.update_layout(
    title='Andamento Prezzi Bitcoin e Transazioni di Vendita',
    xaxis_title='Data',
    yaxis_title='Prezzo (€)',
    hovermode='closest',
    template='plotly_white',
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
    width=1200,
    height=800,
    xaxis=dict(
        rangeslider=dict(visible=True),  # Abilita il range slider
        type="date"  # Imposta l'asse x come asse temporale
    )
)

# Mostra il grafico interattivo
fig.show()

### Grid Search per trovare la miglior combinazione

In [ ]:
def grid_search(param_grid, dati, liquidità_iniziale, bitcoin_iniziali, prezzo_medio_vendite_iniziale):
    """
    Esegue una grid search sulla funzione strategia_vendita per ottimizzare i parametri variabili.
    
    Parameters:
    - param_grid (dict): Dizionario in cui le chiavi sono i nomi dei parametri e i valori sono liste di valori da testare.
    - dati (pd.DataFrame): DataFrame con i dati storici dei prezzi.
    - liquidità_iniziale (float): Liquidità iniziale in euro.
    - bitcoin_iniziali (float): Quantità iniziale di Bitcoin posseduti.
    
    Returns:
    - dict: Risultato con la miglior combinazione e il miglior fattore di rendimento.
    """
    # Genera tutte le combinazioni di parametri
    param_names = list(param_grid.keys())
    param_values = list(param_grid.values())
    all_combinations = list(product(*param_values))
    n_combinations = len(all_combinations)
    
    # Variabili per tracciare il miglior risultato
    miglior_fattore_rendimento = 0
    miglior_iterazione = None
    miglior_combinazione = None

    # Loop attraverso tutte le combinazioni
    for i, values in enumerate(all_combinations, start=1):
        # Crea un dizionario con la combinazione corrente di parametri
        params = dict(zip(param_names, values))
        
        # Esegui la strategia con i parametri correnti
        df_transazioni, prezzi_paracadute = strategia_vendita(
            dati, liquidità_iniziale, bitcoin_iniziali, prezzo_medio_vendite_iniziale, **params
        )
        
        # Calcola il fattore di rendimento
        portafoglio_finale = df_transazioni['valore_totale_portafoglio'].iloc[-1]
        portafoglio_iniziale = liquidità_iniziale + bitcoin_iniziali * dati['Price'].iloc[0]
        fattore_rendimento = portafoglio_finale / portafoglio_iniziale

        # Aggiorna il miglior risultato se il rendimento corrente è superiore
        if fattore_rendimento > miglior_fattore_rendimento:
            miglior_fattore_rendimento = fattore_rendimento
            miglior_iterazione = i
            miglior_combinazione = params

        # Stampa dei risultati dell'iterazione corrente
        print(f"Iterazione {i}/{n_combinations}")
        print(f"  Combinazione testata: {', '.join(f'{k}={v}' for k, v in params.items())}")
        print(f"  Fattore di rendimento: {fattore_rendimento:.4f}")
        print(f"  Miglior fattore di rendimento finora: {miglior_fattore_rendimento:.4f} "
              f"(Iterazione {miglior_iterazione}, Combinazione: {', '.join(f'{k}={v}' for k, v in miglior_combinazione.items())})")
        print("\n")

    # Ritorna il miglior risultato
    return {
        "miglior_fattore_rendimento": miglior_fattore_rendimento,
        "miglior_iterazione": miglior_iterazione,
        "miglior_combinazione": miglior_combinazione
    }

In [ ]:
# Esempio di definizione dei parametri per la grid search
param_grid = {
    "soglia_sopportabile": [0, 0.03, 0.15, 0.2],
    "vendita_rispetto_inizio": [True, False],
    "paracadute_variabile": [True, False],
    "sopportazione_variabile": [True, False],
    "pct_paracadute": [0, 0.08, 0.1, 0.15],
    "A": [0, 1e-4, 1e-3, 1, 2],
    "B": [0, 1e-4, 1e-3, 1, 50],
}

# Esecuzione della grid search
risultato = grid_search(param_grid, dati, liquidità_iniziale, bitcoin_iniziali, prezzo_medio_vendite_iniziale)

# Stampa del miglior risultato
print("Risultato della Grid Search:")
print(f"  Miglior fattore di rendimento: {risultato['miglior_fattore_rendimento']}")
print(f"  Trovato all'iterazione: {risultato['miglior_iterazione']}")
print(f"  Combinazione di parametri ottimale: {', '.join(f'{k}={v}' for k, v in risultato['miglior_combinazione'].items())}")